In [29]:
import pandas as pd

train_df = pd.read_csv('/workspaces/quantchallenge-starter/pranav/data/train.csv')
test_df = pd.read_csv('/workspaces/quantchallenge-starter/pranav/data/test.csv')

print(train_df.head())
print(test_df.head())

   time         A         B         C  ...         M         N        Y1        Y2
0     0  0.207366 -0.159951 -0.634176  ... -0.780237 -0.785965 -0.935902 -0.310081
1     1  0.188828 -0.265508  0.042143  ... -0.161254  0.020401 -0.089707 -0.305374
2     2 -0.144261 -0.577142 -0.214634  ...  0.202726  0.135261 -0.077855 -0.631485
3     3  0.208982 -0.310449  0.513708  ...  0.548993  0.471031  0.941271 -0.535212
4     4  0.093320 -0.358156  0.173188  ...  0.083602  0.003087 -0.039582 -0.490561

[5 rows x 17 columns]
   id   time         A         B  ...         K         L         M         N
0   1  80005 -0.371888 -0.273485  ... -0.369900 -0.378902  0.470166  0.411796
1   2  80006 -0.459598 -0.514915  ... -0.461901 -0.249516 -0.216745 -0.328599
2   3  80007 -0.381609 -0.265023  ... -0.272153 -0.193158 -0.679351 -0.476331
3   4  80008 -0.371423 -0.106279  ...  0.518678 -0.028775 -0.610620 -0.846772
4   5  80009 -0.309393 -0.015144  ...  0.185197 -0.150254 -1.080190 -0.685936

[5 rows x 

In [30]:
train_df.describe()

,time,A,B,C,D,E,F,G,H,I,J,K,L,M,N,Y1,Y2
count,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000
mean,40003.926125,0.068979,-0.004927,-0.000802,0.031913,-0.000292,0.063241,-0.000050,-0.002554,0.070158,-0.000028,-0.000803,0.018752,-0.000636,0.000849,-0.002807,-0.061172
std,23095.035045,1.078271,1.031053,1.010158,1.047274,1.036233,1.069182,1.036270,1.040708,1.064972,1.015265,1.038007,1.057857,1.012744,1.025396,0.970655,0.923702
min,0.000000,-0.642788,-0.771179,-30.068210,-0.899105,-29.530710,-0.737694,-35.715053,-23.598892,-0.762271,-34.806015,-0.729140,-0.734872,-29.342481,-49.869913,-28.918180,-0.850799
25%,20004.750000,-0.374171,-0.451919,-0.417830,-0.500604,-0.475700,-0.415164,-0.419804,-0.437469,-0.395450,-0.432185,-0.405350,-0.438401,-0.386474,-0.366045,-0.387523,-0.497802
50%,40004.500000,-0.264501,-0.285350,-0.003942,-0.279073,-0.006753,-0.236990,-0.004954,0.006673,-0.186564,0.003981,-0.263940,-0.292186,0.004210,0.005000,-0.001591,-0.318091
75%,60004.250000,0.005451,0.061020,0.414435,0.166928,0.458875,0.132404,0.421969,0.444982,0.160427,0.437449,0.033419,0.055187,0.387407,0.367437,0.392470,0.026905
max,80004.000000,24.917549,53.667013,29.312756,28.490356,39.322300,46.837612,22.023252,20.634973,33.604551,19.117766,49.278617,38.849874,27.179463,32.488227,29.859458,34.636039


In [31]:
train_df.shape

(80000, 17)

In [32]:
train_corr = train_df.corr()
train_corr

,time,A,B,C,D,E,F,G,H,I,J,K,L,M,N,Y1,Y2
time,1.000000,-0.389147,0.167514,0.002630,-0.050204,-0.003221,-0.015302,-0.003088,0.002712,0.027639,-0.001808,0.160877,0.128514,0.000954,-0.000042,-0.005645,0.059143
A,-0.389147,1.000000,0.234165,-0.014099,0.401397,-0.001812,0.213891,0.004653,-0.017072,0.233219,-0.024575,0.206646,0.181888,-0.027864,-0.022795,-0.006105,0.484746
B,0.167514,0.234165,1.000000,-0.087635,0.463441,-0.060270,0.425797,-0.065093,-0.093476,0.500890,-0.052194,0.634238,0.526446,-0.095832,-0.085115,-0.094995,0.581859
C,0.002630,-0.014099,-0.087635,1.000000,-0.040376,0.651873,-0.076914,0.657781,0.636027,-0.055286,0.755029,-0.005060,-0.070583,0.662659,0.672503,0.703823,-0.073009
D,-0.050204,0.401397,0.463441,-0.040376,1.000000,0.037757,0.469741,-0.044049,-0.064194,0.458717,-0.069936,0.495356,0.419148,-0.076082,-0.054570,-0.057097,0.611063
E,-0.003221,-0.001812,-0.060270,0.651873,0.037757,1.000000,-0.052012,0.677760,0.616605,-0.059943,0.661936,-0.038899,-0.055095,0.613281,0.529844,0.684297,-0.044583
F,-0.015302,0.213891,0.425797,-0.076914,0.469741,-0.052012,1.000000,-0.059259,-0.019917,0.409510,-0.089096,0.383416,0.371881,-0.087894,-0.076666,-0.073191,0.496667
G,-0.003088,0.004653,-0.065093,0.657781,-0.044049,0.677760,-0.059259,1.000000,0.668732,-0.072203,0.665985,-0.049647,-0.047391,0.660633,0.561462,0.814568,-0.060187
H,0.002712,-0.017072,-0.093476,0.636027,-0.064194,0.616605,-0.019917,0.668732,1.000000,-0.092602,0.661407,-0.082067,-0.075921,0.614654,0.532252,0.707560,-0.086976
I,0.027639,0.233219,0.500890,-0.055286,0.458717,-0.059943,0.409510,-0.072203,-0.092602,1.000000,-0.097424,0.589646,0.468341,-0.096249,0.027320,-0.090226,0.508347


In [34]:
import plotly.express as px
fig = px.imshow(train_corr, color_continuous_scale='RdBu_r')
fig.show()

In [35]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_df['time'], y=train_df['Y1'], mode='lines', name='Y1') )
fig.add_trace(go.Scatter(x=train_df['time'], y=train_df['Y2'], mode='lines', name='Y2'))
# fig.update_layout()
fig

In [43]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import r2_score

# train_df.to_pandas(use_pyarrow_extension_array=False)
print('STAGE 1\n\n\n\n')
def time_features(train_df, base_cols, target_cols=('Y1','Y2'), lags=(1,2,3,5,10,20), rolls=(93,5,10,20,50)):
    df_sorted = train_df.sort_values('time')
    
    for col in list(base_cols)+list(target_cols):
        for k in lags:
            df_sorted[f'{col}_lag{k}']=df_sorted[col].shift(k)
    
    def add_rolls(col):
        s = df_sorted[col].shift(1)
        for w in rolls:
            m = s.rolling(w, min_periods = max(2, int(0.6*w)))
            df_sorted[f'{col}_rmean{w}'] = m.mean()
            df_sorted[f'{col}_rstd{w}'] = m.std()
            df_sorted[f'{col}_rmin{w}'] = m.min()
            df_sorted[f'{col}_rmax{w}'] = m.max()
            
    for col in base_cols:
        add_rolls(col)
        
    
    for col in base_cols:
        df_sorted[f'{col}_diff1'] = df_sorted[col].diff(1)
        
    for P in (24, 48, 96, 256, 1024):
        df_sorted[f'sin_{P}'] = np.sin(2*np.pi*df_sorted['time']/P)
        df_sorted[f'cos_{P}'] = np.cos(2*np.pi*df_sorted['time']/P)
        
    df_sorted = df_sorted.dropna()
    
    return df_sorted


print('STAGE 2\n\n\n')


feature_cols_base = [c for c in train_df.columns if c not in ['time','Y1','Y2']]
df_fe = time_features(train_df, feature_cols_base, target_cols=('Y1','Y2'))

t_train_end, t_valid_end = 60000, 70000
train = df_fe[df_fe['time']<=t_train_end]
valid = df_fe[(df_fe['time']>t_train_end) & (df_fe['time'] <= t_valid_end)]
test = df_fe[df_fe['time']>t_valid_end]

x_cols = [c for c in df_fe.columns if c not in ['time', 'Y1', 'Y2']]
y_cols = ['Y1', 'Y2']

x_train, y1_train, y2_train = train[x_cols], train['Y1'], train['Y2']
x_valid, y1_valid, y2_valid = valid[x_cols], valid['Y1'], valid['Y2']
x_test, y1_test, y2_test = test[x_cols], test['Y1'], test['Y2']

pool_train_y1 = Pool(x_train, y1_train)
pool_valid_y1 = Pool(x_valid, y1_valid)
pool_test_y1 = Pool(x_test, y1_test)


pool_train_y2 = Pool(x_train, y2_train)
pool_valid_y2 = Pool(x_valid, y2_valid)
pool_test_y2 = Pool(x_test, y2_test)

cat_params = dict(loss_function = 'RMSE',
                  learning_rate=0.05,
                  depth=6,
                  l2_leaf_reg=5.0,
                  random_strength=1.0,
                  bootstrap_type='Bernoulli',
                  subsample=0.8,
                  rsm=0.8,
                  iterations=1000,
                  od_type='Iter',
                  od_wait=300,
                  random_seed=42,
                  verbose=False)


model_y1 = CatBoostRegressor(**cat_params)
model_y2 = CatBoostRegressor(**cat_params)


print('STAGE 3\n\n\n')

model_y1.fit(pool_train_y1, eval_set=pool_valid_y1, use_best_model=True, verbose=200)
model_y2.fit(pool_train_y2, eval_set=pool_valid_y2, use_best_model=True, verbose=200)

y1_pred_valid = model_y1.predict(pool_valid_y1)
y2_pred_valid = model_y2.predict(pool_valid_y2)
y1_pred_test = model_y1.predict(pool_test_y1)
y2_pred_test = model_y2.predict(pool_test_y2)

r2_valid_y1 = r2_score(y1_valid, y1_pred_valid)
r2_valid_y2 = r2_score(y2_valid, y2_pred_valid)
r2_test_y1 = r2_score(y1_test, y1_pred_test)
r2_test_y2 = r2_score(y2_test, y2_pred_test)

print(f'Valid R2 - Y1 {r2_valid_y1:.4f}, Y2:{r2_valid_y2:.4f}, Avg: {(r2_valid_y1)/2:.4f}')
print(f'Test R2 - Y1 {r2_test_y1:.4f}, Y2:{r2_test_y2:.4f}, Avg: {(r2_test_y1+r2_test_y2)/2:.4f}')


def top_importances(model, cols, topn=20):
    imp = pd.Series(model.get_feature_importance(type='FeatureImportance'), index=cols)
    return imp.sort_values(ascending=False).head(topn)



print('Top Features for Y1:')
print(top_importances(model_y1, x_cols, 20))
print('Top Features for Y2:')
print(top_importances(model_y2, x_cols, 20))


STAGE 1




STAGE 2





/tmp/ipykernel_33008/1288060834.py:21: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_33008/1288060834.py:22: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_33008/1288060834.py:19: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_33008/1288060834.py:20: Perform

/tmp/ipykernel_33008/1288060834.py:21: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_33008/1288060834.py:22: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_33008/1288060834.py:19: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_33008/1288060834.py:20: Perform

STAGE 3



0:	learn: 0.8669280	test: 0.9653087	best: 0.9653087 (0)	total: 66.5ms	remaining: 1m 6s
200:	learn: 0.3841098	test: 0.5536874	best: 0.5536204 (197)	total: 13.2s	remaining: 52.3s
400:	learn: 0.3604939	test: 0.5503579	best: 0.5503217 (399)	total: 26.1s	remaining: 39s
600:	learn: 0.3434855	test: 0.5485294	best: 0.5484885 (573)	total: 39.4s	remaining: 26.1s
800:	learn: 0.3304545	test: 0.5467574	best: 0.5466969 (785)	total: 52.7s	remaining: 13.1s
999:	learn: 0.3193903	test: 0.5463759	best: 0.5463641 (997)	total: 1m 6s	remaining: 0us

bestTest = 0.5463641377
bestIteration = 997

Shrink model to first 998 iterations.
0:	learn: 0.8139052	test: 0.7976756	best: 0.7976756 (0)	total: 67.3ms	remaining: 1m 7s
200:	learn: 0.3250746	test: 0.3960421	best: 0.3960421 (200)	total: 12.3s	remaining: 49s
400:	learn: 0.2947746	test: 0.3915072	best: 0.3915072 (400)	total: 24.4s	remaining: 36.5s
600:	learn: 0.2766144	test: 0.3887800	best: 0.3887438 (599)	total: 36.8s	remaining: 24.4s
800:	learn: 0.263